In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def read_data(path):
    X = []
    Y = []
    for file_name in os.listdir(path):
        img = Image.open(f'{path}/{file_name}').convert('L')
        img = np.reshape(img, (1, -1))
        label = int(file_name.split('_')[-1].split('.')[0])

        X.append(img)
        Y.append(label)
        
    return np.vstack(X), np.asarray(Y)

In [ ]:
def rand_index_metric(true_labels, pred_labels):
    confusion_matrix = {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    }
    data_size = pred_labels.shape[0]
    for i in range(data_size-1):
        for j in range(i+1, data_size):
            if True:
                if true_labels[i] == true_labels[j]: # positive case
                    if pred_labels[i] == pred_labels[j]:
                        confusion_matrix['TP'] += 1
                    else:
                        confusion_matrix['FP'] += 1
                else: # negative case
                    if pred_labels[i] != pred_labels[j]:
                        confusion_matrix['TN'] += 1
                    else:
                        confusion_matrix['FN'] += 1
    
    rand_indx = (confusion_matrix['TP']+confusion_matrix['TN']) / sum(list(confusion_matrix.values()))
    return rand_indx, confusion_matrix

In [ ]:
def normalize_data(data, mode):
    if mode == 'none':
        return data
    if mode == 'standardScaler':
        return StandardScaler().fit_transform(data)
    if mode == 'range':
        return data/255.
    raise Exception('Invalid mode! mode should be from the following list: ["none", "standardScaler", "range"]')

In [ ]:
data, Y = read_data('/Users/behzad/Downloads/ORL')

In [ ]:
print('X shape:', X.shape)
print('Y shape:', Y.shape)

## KMeans

In [ ]:
mode = 'none'

In [ ]:
X = normalize_data(data, mode)

In [ ]:
kmeans = KMeans(n_clusters=41, random_state=100, n_init=20).fit(X)

In [ ]:
kmeans_predicted_labels = kmeans.labels_

In [ ]:
rand_index_metric(Y, kmeans_predicted_labels)

## DBSCAN

In [ ]:
mode = 'none'

In [ ]:
X = normalize_data(data, mode)

In [ ]:
epsilon = 50
min_samples = 2

In [ ]:
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples).fit(X)

In [ ]:
dbscan_predicted_labels = dbscan.labels_

In [ ]:
rand_index_metric(Y, dbscan_predicted_labels)

## Agglomerative

In [ ]:
mode = 'none'

In [ ]:
X = normalize_data(data, mode)

In [ ]:
n_clusters = 41
linkage = 'complete' # {'complete', 'single', 'average'}

In [ ]:
agglomerative = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage).fit(X)

In [ ]:
agglomerative_predicted_labels = agglomerative.labels_

In [ ]:
rand_index_metric(Y, agglomerative_predicted_labels)

## Enhanced DBSCAN

To have a promising clustering using DBSCAN we have to carefully choose two hyperparameters: 1. MinPts 2. epsilon. During my experience of finding optimal parameters for this assignment, I faced many challenges to find the right epsilon and also figured out its high importance (especially in comparison with MinPts). As I had managed to find an (semi-)optimal value for MinPts, for enhancing performance of DBSCAN I picked the same value for MinPts (in this case 2) and made an effort to find a better epsilon.
Choosing the optimal epsilon has a direct relationship with the outcome. However, finding the optimal epsilon can be overwhelming and tricky. To remedy this, I came with a automated approach which is able to find the optimal epsilon for cases we access to the groundtruth of our data. The process of finding epsilon is as follows:

1. Compute distance (in this case euclidean distance) between each pair of different classes. Now we have a list of distances per each class (say class A) indicating the distances of samples of class A to all other samples from other classes

2. Compute the average distances per each class. Now we have a scalar for each class. E.g., the number associated to class A indicates the average distance of between calss A and other classes' samples

3. Finally, epsilon is computed as the mean of all obtained averages from the previous step. In the end, the value of epsilon is divided by a constant number(2)

In [ ]:
mode = 'standardScaler'

In [ ]:
X = normalize_data(data, mode)

In [ ]:
# step 1
distances = {}
for i in range(Y.shape[0]-1):
    for j in range(i+1, Y.shape[0]):
        distances.setdefault(Y[i], [])
        distances.setdefault(Y[j], [])
        if Y[i] != Y[j]:
            distances[Y[j]].append(np.linalg.norm(X[i]-X[j]))
            distances[Y[i]].append(np.linalg.norm(X[i]-X[j]))

In [ ]:
distances

In [ ]:
# step 2
for key, vals in distances.items():
    distances[key] = np.mean(vals)

In [ ]:
# step 3
epsilon = 0
for key, vals in distances.items():
    epsilon += vals
epsilon /= len(list(distances.keys()))
epsilon /= 2

In [ ]:
min_samples=2

In [ ]:
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples).fit(X)

In [ ]:
dbscan_predicted_labels = dbscan.labels_

In [ ]:
rand_index_metric(Y, dbscan_predicted_labels)